In [9]:
pip install opencv-python matplotlib transformers timm torch torchvision datasets

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [7]:
import cv2
import os

# Create a folder to store the frames
output_folder = "/Users/roupenminassian/Downloads/pedestrian_frames/TRAIN_6352/"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Open the video file
video = cv2.VideoCapture("/Users/roupenminassian/Downloads/IMG_6352.MP4")

# Initialize frame counter
frame_count = 0

# Loop through the video and save each frame
while True:
    ret, frame = video.read()
    
    # Break the loop if we have reached the end of the video
    if not ret:
        break
    
    # Construct the output file name and path
    output_file_path = os.path.join(output_folder, f"frame_{frame_count:04d}.jpg")
    
    # Save the frame as a JPEG file
    cv2.imwrite(output_file_path, frame)
    
    # Print a message to indicate progress
    print(f"Saved {output_file_path}")
    
    # Increment the frame counter
    frame_count += 1

# Release the video capture object
video.release()

print("All frames have been saved.")

Saved /Users/roupenminassian/Downloads/pedestrian_frames/TRAIN_6352/frame_0000.jpg
Saved /Users/roupenminassian/Downloads/pedestrian_frames/TRAIN_6352/frame_0001.jpg
Saved /Users/roupenminassian/Downloads/pedestrian_frames/TRAIN_6352/frame_0002.jpg
Saved /Users/roupenminassian/Downloads/pedestrian_frames/TRAIN_6352/frame_0003.jpg
Saved /Users/roupenminassian/Downloads/pedestrian_frames/TRAIN_6352/frame_0004.jpg
Saved /Users/roupenminassian/Downloads/pedestrian_frames/TRAIN_6352/frame_0005.jpg
Saved /Users/roupenminassian/Downloads/pedestrian_frames/TRAIN_6352/frame_0006.jpg
Saved /Users/roupenminassian/Downloads/pedestrian_frames/TRAIN_6352/frame_0007.jpg
Saved /Users/roupenminassian/Downloads/pedestrian_frames/TRAIN_6352/frame_0008.jpg
Saved /Users/roupenminassian/Downloads/pedestrian_frames/TRAIN_6352/frame_0009.jpg
Saved /Users/roupenminassian/Downloads/pedestrian_frames/TRAIN_6352/frame_0010.jpg
Saved /Users/roupenminassian/Downloads/pedestrian_frames/TRAIN_6352/frame_0011.jpg
Save

In [8]:
import os
import shutil

# Paths to the source folders and destination folder
source_folders = [
    '/Users/roupenminassian/Downloads/pedestrian_frames/TRAIN_0079',
    '/Users/roupenminassian/Downloads/pedestrian_frames/TRAIN_0078',
    '/Users/roupenminassian/Downloads/pedestrian_frames/TRAIN_6352',
    '/Users/roupenminassian/Downloads/pedestrian_frames/TRAIN_6353',
    '/Users/roupenminassian/Downloads/pedestrian_frames/TRAIN_6362',
    '/Users/roupenminassian/Downloads/pedestrian_frames/TRAIN_0069'
]
destination_folder = '/Users/roupenminassian/Downloads/pedestrian_frames/Training File'
os.makedirs(destination_folder, exist_ok=True)

# Counter to keep track of file naming
counter = 1

for folder in source_folders:
    for filename in sorted(os.listdir(folder)):
        # Ensure we're processing a file and not a sub-directory
        if os.path.isfile(os.path.join(folder, filename)):
            # Create the new filename based on the counter
            new_filename = f"frame_{counter:04}.jpg"  # This will format the counter as 001, 002, etc.
            # Move and rename the file to the destination folder
            shutil.move(os.path.join(folder, filename), os.path.join(destination_folder, new_filename))
            # Increment the counter
            counter += 1

print("Files merged and renamed successfully!")


Files merged and renamed successfully!


In [10]:
from PIL import Image
import os
import json

def parse_annotation(annotation_path, image_width, image_height):
    objects = {'id': [], 'area': [], 'bbox': [], 'category': []}
    with open(annotation_path, 'r') as f:
        lines = f.readlines()
        for idx, line in enumerate(lines):
            tokens = line.strip().split(' ')
            category = int(tokens[0])
            center_x, center_y, width, height = map(float, tokens[1:])
            center_x *= image_width
            center_y *= image_height
            width *= image_width
            height *= image_height
            
            # Compute bounding box coordinates
            x1 = center_x - (width / 2)
            y1 = center_y - (height / 2)
            x2 = x1 + width
            y2 = y1 + height
            
            # Clip the coordinates to image boundaries
            x1 = max(0, x1)
            y1 = max(0, y1)
            x2 = min(image_width, x2)
            y2 = min(image_height, y2)
            
            # Recompute width and height after clipping
            width = x2 - x1
            height = y2 - y1
            
            area = width * height
            objects['id'].append(idx)
            objects['area'].append(area)
            objects['bbox'].append([x1, y1, width, height])
            objects['category'].append(category)
    return objects

def create_dataset_v2(image_folder, annotation_folder):
    dataset = []
    image_files = [f for f in os.listdir(image_folder) if f.endswith('.jpg')]
    image_files.sort()
    
    for i, image_file in enumerate(image_files):
        # Construct the annotation filename from the image filename
        annotation_file = image_file.replace('.jpg', '.txt')
        
        # Ensure the annotation file exists
        if not os.path.exists(os.path.join(annotation_folder, annotation_file)):
            print(f"Annotation for {image_file} not found. Skipping.")
            continue
        
        image_path = os.path.join(image_folder, image_file)
        image = Image.open(image_path)
        image_width, image_height = image.size
        
        annotation_path = os.path.join(annotation_folder, annotation_file)
        objects = parse_annotation(annotation_path, image_width, image_height)
        
        dataset_entry = {
            'image_id': i,
            'file_name': image_file,
            'width': image_width,
            'height': image_height,
            'objects': objects
        }
        dataset.append(dataset_entry)
    return dataset

# Paths to image and annotation folders
image_folder = '/Users/roupenminassian/Downloads/pedestrian_frames/Training File'
annotation_folder = '/Users/roupenminassian/Downloads/pedestrian_frames/Annotations File'

dataset_v2 = create_dataset_v2(image_folder, annotation_folder)

# Show first dataset entry
dataset_v2[0] if dataset_v2 else "No valid dataset entries found."

Annotation for frame_0002.jpg not found. Skipping.
Annotation for frame_0003.jpg not found. Skipping.
Annotation for frame_0004.jpg not found. Skipping.
Annotation for frame_0005.jpg not found. Skipping.
Annotation for frame_0006.jpg not found. Skipping.
Annotation for frame_0007.jpg not found. Skipping.
Annotation for frame_0008.jpg not found. Skipping.
Annotation for frame_0009.jpg not found. Skipping.
Annotation for frame_0010.jpg not found. Skipping.
Annotation for frame_0011.jpg not found. Skipping.
Annotation for frame_0012.jpg not found. Skipping.
Annotation for frame_0013.jpg not found. Skipping.
Annotation for frame_0014.jpg not found. Skipping.
Annotation for frame_0015.jpg not found. Skipping.
Annotation for frame_0016.jpg not found. Skipping.
Annotation for frame_0017.jpg not found. Skipping.
Annotation for frame_0018.jpg not found. Skipping.
Annotation for frame_0019.jpg not found. Skipping.
Annotation for frame_0020.jpg not found. Skipping.
Annotation for frame_0021.jpg n

{'image_id': 0,
 'file_name': 'frame_0001.jpg',
 'width': 1280,
 'height': 720,
 'objects': {'id': [0, 1],
  'area': [1399.9910400000003, 1324.9810400255985],
  'bbox': [[254.0, 373.00032, 28.0, 49.99968000000001],
   [228.00064, 366.00012000000004, 24.999679999999984, 52.999919999999975]],
  'category': [0, 0]}}

In [11]:
import json

def save_to_jsonl_v2(dataset, output_file_path):
    """
    Save the dataset to a JSONL (JSON Lines) file in the new format.
    
    Parameters:
    - dataset: List of dataset entries
    - output_file_path: Path to the output JSONL file
    """
    with open(output_file_path, 'w') as f:
        for entry in dataset:
            # Prepare the entry in the desired output format
            output_entry = {
                'file_name': entry['file_name'],  # Use the actual file name
                'image_id': entry['image_id'],
                'width': entry['width'],
                'height': entry['height'],
                'objects': {
                    'id': entry['objects']['id'],
                    'area': entry['objects']['area'],
                    'bbox': entry['objects']['bbox'],
                    'category': entry['objects']['category']
                }
            }
            # Write each entry as a JSON object on a new line
            f.write(json.dumps(output_entry) + '\n')

# Path to the JSONL file
output_file_path = '/Users/roupenminassian/Downloads/pedestrian_frames/metadata.jsonl'
save_to_jsonl_v2(dataset_v2, output_file_path)

# Verify the first few lines from the generated JSONL file
with open(output_file_path, 'r') as f:
    sample_lines_v2 = [next(f) for _ in range(min(20, len(dataset_v2)))]

sample_lines_v2

['{"file_name": "frame_0001.jpg", "image_id": 0, "width": 1280, "height": 720, "objects": {"id": [0, 1], "area": [1399.9910400000003, 1324.9810400255985], "bbox": [[254.0, 373.00032, 28.0, 49.99968000000001], [228.00064, 366.00012000000004, 24.999679999999984, 52.999919999999975]], "category": [0, 0]}}\n',
 '{"file_name": "frame_0103.jpg", "image_id": 102, "width": 1280, "height": 720, "objects": {"id": [0, 1], "area": [1944.0, 2034.9853599744017], "bbox": [[130.99967999999998, 420.99984000000006, 36.0, 54.0], [147.00032, 431.99964, 36.99968000000001, 55.000080000000025]], "category": [0, 0]}}\n',
 '{"file_name": "frame_10013.jpg", "image_id": 1014, "width": 1280, "height": 720, "objects": {"id": [0, 1, 2, 3, 4, 5], "area": [800.0063999999998, 376.99279994880027, 517.9774399488005, 2948.008719974398, 2484.03456, 2330.979839999999], "bbox": [[238.99968, 455.99976, 20.0, 40.00031999999999], [530.00064, 192.99960000000002, 12.999680000000012, 29.000159999999994], [498.99968, 235.9998, 13.

In [13]:
import json
import shutil
import os

def copy_annotated_images(jsonl_path, src_folder, dest_folder):
    # Ensure the destination folder exists
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)
    
    # Open the jsonl file and iterate through each line
    with open(jsonl_path, 'r') as f:
        for line in f:
            data = json.loads(line)
            file_name = data['file_name']
            src_path = os.path.join(src_folder, file_name)
            dest_path = os.path.join(dest_folder, file_name)
            
            # Check if the file exists in the source folder
            if os.path.exists(src_path):
                # Copy the file to the destination folder
                shutil.copy2(src_path, dest_path)

# Path to the jsonl file
jsonl_path = '/Users/roupenminassian/Downloads/pedestrian_frames/metadata.jsonl'
# Source folder containing the images
src_folder = '/Users/roupenminassian/Downloads/pedestrian_frames/Training File/'
# Destination folder where the images will be copied
dest_folder = '/Users/roupenminassian/Downloads/pedestrian_frames/Train/'

# Call the function
copy_annotated_images(jsonl_path, src_folder, dest_folder)

In [14]:
from datasets import load_dataset

folderPath = "/Users/roupenminassian/Downloads/pedestrian_frames/Training"

dataset = load_dataset("imagefolder", data_dir=folderPath)
dataset.push_to_hub("roupenminassian/pedestrian-dataset")

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Resolving data files: 100%|██████████| 79/79 [00:00<00:00, 208396.24it/s]
Extracting data files: 0it [00:00, ?it/s]
Generating train split: 78 examples [00:00, 1542.11 examples/s]
Pushing dataset shards to the dataset hub: 100%|██████████| 1/1 [00:12<00:00, 12.98s/it]
